In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
getnet_files_transaction = sorted(glob.glob(r'G:\My Drive\RESPALDO-PC\GETNET\INFO GETNET - DB\*.tsv', recursive=False))
getnet_files_transaction

['G:\\My Drive\\RESPALDO-PC\\GETNET\\INFO GETNET - DB\\GETNET DETALLE_BY_TXN_ID.tsv']

In [3]:
getnet_transaction = pd.concat([pd.read_csv(f, sep='\t',dtype=object)
                         .assign(file_name=os.path.basename(f)) for f in getnet_files_transaction])
getnet_transaction.columns = getnet_transaction.columns.str.strip().str.lower()
getnet_transaction['bin_getnet'] = getnet_transaction['card_number'].str[:7].astype(str)
getnet_transaction['bin_getnet'] = getnet_transaction['bin_getnet'].str.replace('-','').astype(str)
getnet_transaction['last4_getnet'] = getnet_transaction['card_number'].str[-4:].astype(str)
getnet_transaction['transaction_amount'] = getnet_transaction['transaction_amount'].str.replace(',','').astype(float)
getnet_transaction['auth_code'] = getnet_transaction['auth_code'].str.replace("'",'').astype(str)
getnet_transaction['custom_fields_1'] = getnet_transaction['custom_fields_1'].str.zfill(9)
getnet_transaction['INDEX_AUTH_CODE'] = (getnet_transaction['seller'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['bin_getnet'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['last4_getnet'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['auth_code'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['transaction_amount'].astype(str).replace(' ', '',regex=True))
getnet_transaction['INDEX_CUST_FIELD'] = (getnet_transaction['seller'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['bin_getnet'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['last4_getnet'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['custom_fields_1'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['transaction_amount'].astype(str).replace(' ', '',regex=True))
getnet_transaction['INDEX_AUTH_CODE_2'] = (getnet_transaction['seller'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['last4_getnet'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['auth_code'].astype(str).replace(' ', '',regex=True)+'_'
                               +getnet_transaction['transaction_amount'].astype(str).replace(' ', '',regex=True))
getnet_transaction.head()

,chain_id,merchant,merchant_name,seller,seller_name,terminal_id,number_account,arn,app_id,auth_code,...,metadata_bronze_filename,record_id,spark_metadata_app_id,spark_metadata_batch_id,file_name,bin_getnet,last4_getnet,INDEX_AUTH_CODE,INDEX_CUST_FIELD,INDEX_AUTH_CODE_2
0,NaN,BLI120726UF6,PAYCLIP SRL CV,8991118,PAYCLIP*ALIM Y BEBIDAS,PMX1118A,'014180655068329767,'74512983249444392217257,mit,124147,...,202309071154ID100003settransactiongmr.csv,407b32d0-3027-5acc-841e-8d61eb7980ee,e79c4f38-b1c9-40f1-9d06-4f2f53d0c203,41,GETNET DETALLE_BY_TXN_ID.tsv,481588,7479,8991118_481588_7479_124147_385.0,8991118_481588_7479_100001547655_385.0,8991118_7479_124147_385.0
1,NaN,BLI120726UF6,PAYCLIP SRL CV,8991118,PAYCLIP*ALIM Y BEBIDAS,8991118,'014180655068329767,'75177133249219189911188,by_pass,774620,...,202309071154ID100003settransactiongmr.csv,cc62ae49-88b0-532d-8049-3467776a3785,e79c4f38-b1c9-40f1-9d06-4f2f53d0c203,41,GETNET DETALLE_BY_TXN_ID.tsv,546637,9043,8991118_546637_9043_774620_4197.0,8991118_546637_9043_001547654_4197.0,8991118_9043_774620_4197.0
2,NaN,BLI120726UF6,PAYCLIP SRL CV,9044851,CLIPMX*NATURAL,9044851,'014180655068329767,'74512983249470190448513,by_pass,311463,...,202309071154ID100003settransactiongmr.csv,e02d07e0-ec86-52c0-8e03-fef395ac76e9,e79c4f38-b1c9-40f1-9d06-4f2f53d0c203,41,GETNET DETALLE_BY_TXN_ID.tsv,415231,4460,9044851_415231_4460_311463_182.0,9044851_415231_4460_001547659_182.0,9044851_4460_311463_182.0
3,NaN,BLI120726UF6,PAYCLIP SRL CV,8991133,PAYCLIP*TDAS AUT MINISUP,PMX1133A,'014180655068329767,'74512983249443527824920,mit,954075,...,202309071154ID100003settransactiongmr.csv,597c4e61-8f6a-5605-8053-0a1748b292d3,e79c4f38-b1c9-40f1-9d06-4f2f53d0c203,41,GETNET DETALLE_BY_TXN_ID.tsv,401543,9561,8991133_401543_9561_954075_162.24,8991133_401543_9561_100001547652_162.24,8991133_9561_954075_162.24
4,NaN,BLI120726UF6,PAYCLIP SRL CV,8991118,PAYCLIP*ALIM Y BEBIDAS,8991118,'014180655068329767,'74512983249211989911182,by_pass,01812D,...,202309071154ID100003settransactiongmr.csv,c4f894ec-628d-52b4-b529-8ac46c79abd2,e79c4f38-b1c9-40f1-9d06-4f2f53d0c203,41,GETNET DETALLE_BY_TXN_ID.tsv,440066,5786,8991118_440066_5786_01812D_672.75,8991118_440066_5786_001547658_672.75,8991118_5786_01812D_672.75


In [4]:
getnet_transaction.shape

(3087231, 69)

In [5]:
getnet_transaction.dtypes

chain_id             object
merchant             object
merchant_name        object
seller               object
seller_name          object
                      ...  
bin_getnet           object
last4_getnet         object
INDEX_AUTH_CODE      object
INDEX_CUST_FIELD     object
INDEX_AUTH_CODE_2    object
Length: 69, dtype: object

In [6]:
getnet_transaction.columns

Index(['chain_id', 'merchant', 'merchant_name', 'seller', 'seller_name',
       'terminal_id', 'number_account', 'arn', 'app_id', 'auth_code',
       'transaction_type', 'transaction_mode', 'pos_entry_mode',
       'transaction_channel', 'transaction_date', 'transaction_time',
       'time_zone', 'settlement_date', 'movement_number', 'transaction_amount',
       'transaction_currency', 'settlement_amount', 'settlement_currency',
       'total_commission_amount', 'total_vat_tax_commision_amount',
       'mdr_commission_amount', 'commision_rate', 'vat_tax_amount',
       'total_commission_dia', 'markup_rate', 'markup_amount',
       'transactional_cost_fee_rate', 'transactional_cost_fee_amount',
       'interchange_rate', 'total_interchange_amount', 'assessment_rate',
       'total_assessment_amount', 'other_cost_incomes_rate',
       'other_cost_incomes_amount',
       'vat_tax_commission_other_cost_income_amount',
       'payment_special_indicator', 'installment_flag_ind',
       'inst

# CHECKS

In [7]:
ticket_zendesk = pd.read_csv('DISPUTAS GETNET 2023.09.08.csv',dtype={'AMOUNT':float,'CLAVE_AUTORIZACION': object,'BIN':object
                                                  ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object})
ticket_zendesk.columns = ticket_zendesk.columns.str.strip().str.lower()
ticket_zendesk['INDEX_ORDER_ID'] = (ticket_zendesk['affiliation'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['bin'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['last4'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['amount'].astype(str).replace(' ', '',regex=True))
ticket_zendesk['INDEX_CUST_FIELD'] = (ticket_zendesk['affiliation'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['bin'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['last4'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['amount'].astype(str).replace(' ', '',regex=True))
ticket_zendesk['INDEX_AUTH_CODE'] = (ticket_zendesk['affiliation'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['bin'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['last4'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['amount'].astype(str).replace(' ', '',regex=True))
ticket_zendesk['INDEX_AUTH_CODE_2'] = (ticket_zendesk['affiliation'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['last4'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                                     +ticket_zendesk['amount'].astype(str).replace(' ', '',regex=True))
ticket_zendesk.head()

,merchant_id,uuid,transaction_id,merchant_name,order_id,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,...,msi,device_id,clip_reader,status_msg,issuer,brand,INDEX_ORDER_ID,INDEX_CUST_FIELD,INDEX_AUTH_CODE,INDEX_AUTH_CODE_2
0,88646ffe-6ede-47b4-a269-889c4d14b3ff,300d514c-8a8f-456d-b2d1-0adeafccc678,558a3251-1acb-46a8-8300-f3356feb449e,LOS REMEDIOS NATURISTA,252290843369,8991133,CARD_PRESENT,62333351,5912,2023-08-12,...,0.00,PP36812120014028,Clip Pro,Error. Cancel requested on pending payment.,MONZO BANK LIMITED,MASTERCARD,8991133_535522_0499_252290843369_1500.0,8991133_535522_0499_nan_1500.0,8991133_535522_0499_nan_1500.0,8991133_0499_nan_1500.0


In [8]:
ticket_zendesk.shape

(1, 30)

# CRUCE_2: DATA CLIP VS GETNET

In [14]:
cruce_1 = ticket_zendesk.merge(getnet_transaction[['INDEX_AUTH_CODE','auth_code','bin_getnet','last4_getnet','arn',
                                                   'transaction_type','transaction_date','settlement_date','seller',
                                                   'transaction_amount']],on='INDEX_AUTH_CODE',how='left')
cruce_1.head()

,merchant_id,uuid,transaction_id,merchant_name,order_id,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,...,INDEX_AUTH_CODE_2,auth_code,bin_getnet,last4_getnet,arn,transaction_type,transaction_date,settlement_date,seller,transaction_amount
0,88646ffe-6ede-47b4-a269-889c4d14b3ff,300d514c-8a8f-456d-b2d1-0adeafccc678,558a3251-1acb-46a8-8300-f3356feb449e,LOS REMEDIOS NATURISTA,252290843369,8991133,CARD_PRESENT,62333351,5912,2023-08-12,...,8991133_0499_nan_1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
cruce_1.shape

(1, 39)

In [16]:
cruce_1_final = cruce_1[['merchant_id','transaction_id','status_code','order_id','arn','auth_code',
                          'seller','transaction_date','transaction_type','settlement_date','transaction_amount']]
cruce_1_final.head()                 

,merchant_id,transaction_id,status_code,order_id,arn,auth_code,seller,transaction_date,transaction_type,settlement_date,transaction_amount
0,88646ffe-6ede-47b4-a269-889c4d14b3ff,558a3251-1acb-46a8-8300-f3356feb449e,-5,252290843369,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
cruce_1_final.shape

(1, 11)

In [18]:
writer = pd.ExcelWriter('DATA FINAL DISPUTAS GETNET-2 2023.09.08.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
cruce_1_final.to_excel(writer, sheet_name='MATCH_FINAL', index=False)
cruce_1.to_excel(writer, sheet_name='MATCH', index=False)
writer.close()

# CHECKS PUNTUALES

In [19]:
# select columns by name
check_txn_puntual = getnet_transaction[(getnet_transaction['INDEX_AUTH_CODE_2'].isin([
'8991133_0499_AP1RDI_1500.0'
]))]
check_txn_puntual.head()

,chain_id,merchant,merchant_name,seller,seller_name,terminal_id,number_account,arn,app_id,auth_code,...,metadata_bronze_filename,record_id,spark_metadata_app_id,spark_metadata_batch_id,file_name,bin_getnet,last4_getnet,INDEX_AUTH_CODE,INDEX_CUST_FIELD,INDEX_AUTH_CODE_2
614646,NaN,BLI120726UF6,PAYCLIP SRL CV,8991133,PAYCLIP*TDAS AUT MINISUP,PMX1133A,'014180655068329767,'75177133224000688966611,mit,AP1RDI,...,202308141100ID100003settransactiongmr.csv,95001bf4-a597-599c-9ed1-56438751856b,c20fb4d7-1f6b-42bd-9919-691bd69aa2b7,23,GETNET DETALLE_BY_TXN_ID.tsv,535522,0499,8991133_535522_0499_AP1RDI_1500.0,8991133_535522_0499_100000848528_1500.0,8991133_0499_AP1RDI_1500.0


In [20]:
check_txn_puntual.shape

(1, 69)

In [21]:
check_txn_puntual_final = check_txn_puntual[['INDEX_AUTH_CODE','INDEX_CUST_FIELD','INDEX_AUTH_CODE_2','arn','auth_code',
                                             'seller','transaction_date','transaction_type','settlement_date',
                                             'transaction_amount']]
check_txn_puntual_final.head()

,INDEX_AUTH_CODE,INDEX_CUST_FIELD,INDEX_AUTH_CODE_2,arn,auth_code,seller,transaction_date,transaction_type,settlement_date,transaction_amount
614646,8991133_535522_0499_AP1RDI_1500.0,8991133_535522_0499_100000848528_1500.0,8991133_0499_AP1RDI_1500.0,'75177133224000688966611,AP1RDI,8991133,2023-08-12,Sale,2023-08-14,"1,500.00"


In [22]:
check_txn_puntual_final.shape

(1, 10)

In [23]:
writer = pd.ExcelWriter('DATA FINAL PUNT DISPUTAS GETNET 2023.09.08.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
check_txn_puntual_final.to_excel(writer, sheet_name='MATCH_FINAL', index=False)
#check_txn.to_excel(writer, sheet_name='MATCH', index=False)
writer.close()